In [14]:
import os
import numpy as np
import pandas as pd
import pretty_midi
from music21 import converter, chord
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Data Collection | Pre-Processing | Feature Extraction

In [ ]:
# Path to the dataset folder
MIDI_FOLDER = "./dataset" 
CSV_FOLDER = "./features_csv"

# Ensure the CSV folder exists
os.makedirs(CSV_FOLDER, exist_ok=True)

def extract_features(midi_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        features = {}

        # Extract global features like tempo and number of instruments
        features['tempo'] = np.mean(midi_data.get_tempo_changes()[1])
        features['num_instruments'] = len(midi_data.instruments)

        # Extract note-level features for each instrument
        notes = []
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes.append((note.pitch, note.start, note.end, note.velocity))

        # Convert notes to a DataFrame to facilitate further analysis
        if notes:
            df_notes = pd.DataFrame(notes, columns=['Pitch', 'Start', 'End', 'Velocity'])
            features['avg_pitch'] = df_notes['Pitch'].mean()
            features['std_pitch'] = df_notes['Pitch'].std()
            features['avg_velocity'] = df_notes['Velocity'].mean()
            features['std_velocity'] = df_notes['Velocity'].std()

        # Analyze chords using music21
        score = converter.parse(midi_path)
        chords = [ch for ch in score.chordify().recurse() if isinstance(ch, chord.Chord)]
        pitches = [p for ch in chords for p in ch.pitches]
        features['num_chords'] = len(chords)
        features['unique_pitches'] = len(set(pitches))

        return features
    except Exception as e:
        print(f"Error processing {midi_path}: {e}")
        return {}

def process_files():
    csv_path = os.path.join(CSV_FOLDER, 'features.csv')
    composers = ['Bach', 'Beethoven', 'Chopin', 'Mozart']
    all_features = []

    for composer in composers:
        composer_path = os.path.join(MIDI_FOLDER, composer)
        midi_files = [f for f in os.listdir(composer_path) if f.endswith('.mid')]

        for filename in tqdm(midi_files, desc=f"Processing {composer} files"):
            midi_path = os.path.join(composer_path, filename)
            features = extract_features(midi_path)
            if features:
                features['filename'] = filename
                features['composer'] = composer
                all_features.append(features)

    # Save all features to a CSV file
    df = pd.DataFrame(all_features)
    df.to_csv(csv_path, index=False)
    print("Features saved to:", csv_path)

if __name__ == '__main__':
    process_files()

# Model Building

In [ ]:
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Model Training

In [ ]:
# Load the features from CSV
df = pd.read_csv('./features_csv/features.csv')

# Assuming 'composer' is the target variable
X = df.drop(['composer', 'filename'], axis=1).values
y = pd.get_dummies(df['composer']).values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the model
model = build_model((1, X_train.shape[2]), y_train.shape[1])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Model Evaluation

In [ ]:
# Predict classes
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=df['composer'].unique()))

# Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', xticklabels=df['composer'].unique(), yticklabels=df['composer'].unique())
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Model Optimization

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    return build_model((1, X_train.shape[2]), y_train.shape[1])

# Wrap Keras model so it can be used by scikit-learn
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adam']
batch_size = [32, 64, 128]
epochs = [20, 50, 100]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print best results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))